# Lab 1 - Exercise 1 - Create connection and run simple queries


In [ ]:
############################## CREATE MongoDB Atlas connection ##############################
import pymongo
from pprint import pprint

import sys
sys.path.append("../config.py")
import config as cfg

# Connect to Atlas
client = pymongo.MongoClient(cfg.CONNECTIONSTRING)
db = client[cfg.DB]
coll = db[cfg.COLL]

############################## Connection TEST ##############################
# Show the fields "documentNumber", "country" and "segmentTypeLocal" of 2 documents WHERE the field "segmentTypeLocal" is not NULL
#
query = {"firstName": {'$nin': ["null", ""]}}
project = {"_id": 0, "firstName": 1, "documentNumber": 1, "country": 1}

cursor = coll.find(query,project).limit(2)

pprint('------- 2 samples of customers: ')
for document in cursor:
    pprint(document)

# Count the number of documents in the collection
count = coll.count_documents({})
print('------- Total number of documents in the collection: ')
print(count)



# Lab 1 - Exercise 2 - Run simple Aggregation Pipeline 

In [ ]:
############################## simple AGGREGATON pipeline #############################
from bson.son import SON

pipeline = [
    {"$match": {"lastName": 'Evelyn'}},
    {"$sort": SON([("documentNumber", -1)])},
    {"$project": {"_id": 0, "firstName": 1,"lastName": 1, "documentNumber": 1, "country": 1}},
    {"$limit": 5}
]

print('------- 5 samples of customers with lastName Evelyn and sort by documentNumber: ')
cursor = coll.aggregate(pipeline)
for document in cursor:
    pprint(document)
#pprint(list(coll.aggregate(pipeline)))

# Lab 1 - Challenge

Modify the previous Aggregation Pipeline to find the Customer with "lastName=Evelyn" AND "country=C1" and send me the first 4 docs with the field "segmentLocal" included.



# Lab 2 - Exercise 1: Documents fields conversion, count documents to change

In [ ]:
############################## find docs to change ##############################

cursor = coll.find({'companyStartDate': {'$exists': True, '$type': 'string'}},{"documentNumber": 1, "country": 1, "companyStartDate": 1}).limit(2)
for document in cursor:
    pprint(document)
count = coll.count_documents({'companyStartDate': {'$exists': True, '$type': 'string'}})
print('Number of documents to change: ' + str(count))

# Lab 2 - Exercise 2: Document fields conversion, from String to Date

In [ ]:
############################## change String fields to Date fields  #############################
#result3 = coll.update_many({'companyStartDate': {'$exists': True, '$type': 'string'}}, [{'$set': {'companyStartDate': { '$toDate': '$companyStartDate'}}}])
#print(result3.raw_result)

#result4 = coll.update_many({'companyEndDate': {'$exists': True, '$type': 'string'}}, [{'$set': {'companyEndDate': { '$toDate': '$companyEndDate'}}}])
#print(result4.raw_result)

result = coll.update_many({'$or': [{'companyStartDate': {'$exists': True, '$type': 'string'}},{'companyEndDate': {'$exists': True, '$type': 'string'}}]}, [{'$set': {'companyStartDate': { '$toDate': '$companyStartDate'}}},{'$set': {'companyEndDate': { '$toDate': '$companyEndDate'}}}])
print(result.raw_result)


In [ ]:
############################## check Date fields ##############################

cursor = coll.find({'companyStartDate': {'$exists': True, '$type': 'date'}},{"documentNumber": 1, "country": 1, "companyStartDate": 1}).limit(2)
for document in cursor:
    pprint(document)

# Lab 2 - Exercise 3: Find Customers between dates (note that no index is used)

In [ ]:
############################## find docs to change ##############################
from datetime import datetime

# convert your date string to datetime object
start = datetime(1995, 1, 20, 7, 51, 59)
end = datetime(1996, 1, 20, 7, 52, 59)

cursor = coll.find({"companyStartDate":{'$gte':start,'$lt':end}},{"documentNumber": 1, "country": 1, "companyStartDate": 1}).limit(2)
for document in cursor:
    pprint(document)

count = coll.count_documents({"companyStartDate":{'$gte':start,'$lt':end}})
pprint(count)


# Lab 2 - Exercise 4: Create a new field for all documents (498.825) with the days in the company

In [ ]:
############################## create NEW FIELD (duration_days) with days in the company  ##############################
cursor = coll.update_many(
      {},
      [ { "$set" : {
        "duration_days" : {'$dateDiff': {'startDate': '$companyStartDate','endDate': '$companyEndDate','unit': 'day'}}
      } } ]
    )
pprint(cursor.raw_result)

# Lab 2 - Exercise 5: Operator $group from Aggregation Pipeline

In [ ]:
result = coll.aggregate([
    {
        '$set': {
            'companyEndDate': {
                '$toDate': '$companyEndDate'
            }, 
            'companyStartDate': {
                '$toDate': '$companyStartDate'
            }
        }
    }, {
        '$group': {
            '_id': {
                '$year': '$companyStartDate'
            }, 
            'numberofcompanies': {
                '$sum': 1
            }
        }
    }, {
        '$sort': {
            '_id': -1
        }
    }, {
        '$set': {
            'creationyear': '$_id'
        }
    }, {
        '$project': {
            'creationyear': 1, 
            'numberofcompanies': 1, 
            '_id': 0
        }
    }
])
for doc in result:
    print(doc)


# Lab 2 - Challenge

Count the number of Open accounts per documentNumber and send me the first two with more Open accounts.
It should look like:

```
{
  "_id": {
    "accountsList_status": "Open",
    "documentNumber": "0016124399K"
  },
  "counter": 240
},
{
  "_id": {
    "accountsList_status": "Open",
    "documentNumber": "00234740636"
  },
  "counter": 240
}
```

CLUE: https://www.mongodb.com/docs/manual/reference/operator/aggregation/unwind/

## *** Go back for Lab 3 instructions in README.md file***

## *** BONUS : Graph Capabilities ***
MongoDB provides graph and tree traversal capabilities with its $graphLookup stage in the aggregation pipeline.
In this section we are going to modify the dataset to create relation between documents and then execute an aggregation pipeline using $graphLookup to show these interactions

The graphical representation look like the following image:

<img src="./img/graphLookup.png" alt="drawing" width="800"/>

In [ ]:
############################## Modify the dataset to create relationships  ##############################
document = coll.find_one({'$and' : [{'segmentLocal' : 'GRANDES EMPRESAS'}, {'companyName': 'COML KAUFMANN S A'}, {'country' : 'CL'}]})
cursor = coll.update_one(
      {'_id' : document['_id']},
      [ { "$set" : {
        "companyName" : "VEHICLES SA",
        "tradeName" : 'VOLVO'
      } } ]
    )
pprint(cursor.raw_result)

document = coll.find_one({'$and' : [{'segmentLocal' : 'GRANDES EMPRESAS'}, {'companyName': 'COML KAUFMANN S A'}, {'country' : 'CL'}]})
cursor = coll.update_one(
      {'_id' : document['_id']},
      [ { "$set" : {
        "companyName" : "VEHICLES SA",
        "tradeName" : 'MERCEDES BENZ'
      } } ]
    )
pprint(cursor.raw_result)

document = coll.find_one({'$and' : [{'segmentLocal' : 'GRANDES EMPRESAS'}, {'companyName': 'COML KAUFMANN S A'}, {'country' : 'CL'}]})
cursor = coll.update_one(
      {'_id' : document['_id']},
      [ { "$set" : {
        "companyName" : "MERCEDES BENZ",
        "tradeName" : 'SMART'
      } } ]
    )
pprint(cursor.raw_result)

document = coll.find_one({'$and' : [{'segmentLocal' : 'GRANDES EMPRESAS'}, {'companyName': 'COML KAUFMANN S A'}, {'country' : 'CL'}]})
cursor = coll.update_one(
      {'_id' : document['_id']},
      [ { "$set" : {
        "companyName" : "VOLVO",
        "tradeName" : 'VOLVO TRUCKS'
      } } ]
    )
pprint(cursor.raw_result)

document = coll.find_one({'$and' : [{'segmentLocal' : 'GRANDES EMPRESAS'}, {'companyName': 'COML KAUFMANN S A'}, {'country' : 'CL'}]})
cursor = coll.update_one(
      {'_id' : document['_id']},
      [ { "$set" : {
        "companyName" : "VOLVO TRUCKS",
        "tradeName" : 'VOLVO BIG TRUCKS'
      } } ]
    )
pprint(cursor.raw_result)

In [ ]:
############################## Execute $graphLookup  ##############################

aggregation_pipeline = [
    {
        '$project': {
            'firstName': 1, 
            'lastName': 1, 
            'companyName': 1, 
            'tradeName': 1
        }
    }, {
        '$graphLookup': {
            'from': 'Customer', 
            'startWith': '$tradeName', 
            'connectFromField': 'tradeName', 
            'connectToField': 'companyName', 
            'as': 'tradesAs', 
            'maxDepth': 2
        }
    }, {
        '$project': {
            'companyName': 1, 
            'tradeName': 1, 
            'tradesAs.tradeName': 1
        }
    }, {
        '$match': {
            'companyName': 'VEHICLES SA'
        }
    }
]
result = coll.aggregate(aggregation_pipeline)
for doc in result:
    print(doc)